Importing All the Necessary Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import torch
from torchvision import datasets, transforms, models
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F
from datetime import datetime
import shutil
from google.colab import drive
import torchvision.transforms.functional as TF
from PIL import Image

Mounting Google Drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


Defining the path of the dataset

In [ ]:
google_drive_path = '/content/drive/MyDrive/366 Dataset/Plant_leave_diseases_dataset_without_augmentation'
colab_path = '/content/dataset5'
shutil.copytree(google_drive_path, colab_path)

'/content/dataset5'

Unzip if Needed

In [ ]:
!unzip -q /content/drive/MyDrive/dataset1.zip -d /content/

unzip:  cannot find or open /content/drive/MyDrive/dataset1.zip, /content/drive/MyDrive/dataset1.zip.zip or /content/drive/MyDrive/dataset1.zip.ZIP.


Resize the image

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to 224x224
    transforms.ToTensor(),           # Convert to tensor
    transforms.Normalize(            # Normalize the image
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [ ]:
dataset = datasets.ImageFolder(colab_path, transform=transform)
print("Number of classes:", len(dataset.classes))

Number of classes: 39


In [ ]:
print(f"Total dataset size: {len(dataset)}")
print(f"Classes: {dataset.classes}")

Total dataset size: 45307
Classes: ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Background_without_leaves', 'Blueberry___healthy', 'Cherry___Powdery_mildew', 'Cherry___healthy', 'Corn___Cercospora_leaf_spot Gray_leaf_spot', 'Corn___Common_rust', 'Corn___Northern_Leaf_Blight', 'Corn___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot'

In [ ]:
indices = list(range(len(dataset)))
split = int(np.floor(0.85 * len(dataset)))  # train_size
validation = int(np.floor(0.70 * split))  # validation
np.random.shuffle(indices)
train_indices, validation_indices, test_indices = (
    indices[:validation],
    indices[validation:split],
    indices[split:],
)

In [ ]:
train_sampler = SubsetRandomSampler(train_indices)
validation_sampler = SubsetRandomSampler(validation_indices)
test_sampler = SubsetRandomSampler(test_indices)

In [ ]:
class CNN(nn.Module):
    def __init__(self, K):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2),
        )
        self.dense_layers = nn.Sequential(
            nn.Dropout(0.4),
            nn.Linear(50176, 1024),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(1024, K),
        )

    def forward(self, X):
        out = self.conv_layers(X)
        out = out.view(-1, 50176)
        out = self.dense_layers(out)
        return out

In [ ]:
def check_gradients(model):
    grads = []
    for param in model.parameters():
        if param.grad is not None:
            grads.append(param.grad.abs().sum().item())
    return np.sum(grads)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
targets_size = len(dataset.class_to_idx)
model = CNN(targets_size)
model.to(device)

CNN(
  (conv_layers): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)

In [ ]:
from torchsummary import summary
summary(model, (3, 224, 224))
criterion = nn.CrossEntropyLoss()  # this includes softmax + cross entropy loss
optimizer = torch.optim.Adam(model.parameters())

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]             896
              ReLU-2         [-1, 32, 224, 224]               0
       BatchNorm2d-3         [-1, 32, 224, 224]              64
            Conv2d-4         [-1, 32, 224, 224]           9,248
              ReLU-5         [-1, 32, 224, 224]               0
       BatchNorm2d-6         [-1, 32, 224, 224]              64
         MaxPool2d-7         [-1, 32, 112, 112]               0
            Conv2d-8         [-1, 64, 112, 112]          18,496
              ReLU-9         [-1, 64, 112, 112]               0
      BatchNorm2d-10         [-1, 64, 112, 112]             128
           Conv2d-11         [-1, 64, 112, 112]          36,928
             ReLU-12         [-1, 64, 112, 112]               0
      BatchNorm2d-13         [-1, 64, 112, 112]             128
        MaxPool2d-14           [-1, 64,

In [ ]:
def batch_gd(model, criterion, train_loader, validation_loader, epochs):
    train_losses = np.zeros(epochs)
    validation_losses = np.zeros(epochs)

    for e in range(epochs):
        t0 = datetime.now()
        train_loss = []
        model.train()  # Ensure model is in training mode
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            output = model(inputs)
            loss = criterion(output, targets)
            train_loss.append(loss.item())
            loss.backward()
            optimizer.step()
        train_loss = np.mean(train_loss)
        validation_loss = []
        model.eval()  # Ensure model is in evaluation mode
        with torch.no_grad():
            for inputs, targets in validation_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                output = model(inputs)
                loss = criterion(output, targets)
                validation_loss.append(loss.item())
        validation_loss = np.mean(validation_loss)
        train_losses[e] = train_loss
        validation_losses[e] = validation_loss
        dt = datetime.now() - t0
        print(f"Epoch: {e+1}/{epochs} Train_loss: {train_loss:.3f} Validation_loss: {validation_loss:.3f} Duration: {dt}")
    return train_losses, validation_losses

In [ ]:
initial_grad_sum = check_gradients(model)
print(f"Initial gradient sum: {initial_grad_sum}")

Initial gradient sum: 0.0


In [ ]:
batch_size = 64
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=validation_sampler)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)

In [ ]:
train_losses, validation_losses = batch_gd(model, criterion, train_loader, validation_loader, 5)
torch.save(model.state_dict(), '/content/drive/MyDrive/Trained Models/trained_last.pt')

Epoch: 1/5 Train_loss: 0.429 Validation_loss: 0.301 Duration: 0:03:33.404482
Epoch: 2/5 Train_loss: 0.381 Validation_loss: 0.318 Duration: 0:03:32.737165
Epoch: 3/5 Train_loss: 0.359 Validation_loss: 0.297 Duration: 0:03:33.231825
Epoch: 4/5 Train_loss: 0.330 Validation_loss: 0.331 Duration: 0:03:33.587551
Epoch: 5/5 Train_loss: 0.287 Validation_loss: 0.314 Duration: 0:03:33.413228


In [ ]:
plt.plot(train_losses, label='train_loss')
plt.plot(validation_losses, label='validation_loss')
plt.xlabel('Number of Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
def accuracy(loader, model, device):
    model.eval()
    n_correct = 0
    n_total = 0
    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predictions = torch.max(outputs, 1)
            n_correct += (predictions == targets).sum().item()
            n_total += targets.shape[0]
    return n_correct / n_total


In [ ]:
train_acc = accuracy(train_loader, model, device)
test_acc = accuracy(test_loader, model, device)
validation_acc = accuracy(validation_loader, model, device)
print(f"Train Accuracy: {train_acc}\nTest Accuracy: {test_acc}\nValidation Accuracy: {validation_acc}")

Train Accuracy: 0.969173127573543
Test Accuracy: 0.9208474326908931
Validation Accuracy: 0.9265991517354799


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns

def plot_confusion_matrix(loader, model, device, classes):
    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predictions = torch.max(outputs, 1)
            all_preds.extend(predictions.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    cm = confusion_matrix(all_targets, all_preds)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    disp.plot(cmap=plt.cm.Blues)
    plt.xticks(rotation=90)
    plt.show()

# Plot confusion matrix for test data
plot_confusion_matrix(test_loader, model, device, dataset.classes)

In [ ]:
def single_prediction(image_path, model, data, device):
    model.eval()
    image = Image.open(image_path)
    image = image.resize((224, 224))
    input_data = TF.to_tensor(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(input_data)
    index = torch.argmax(output, dim=1).item()
    print("Original:", image_path[12:-4])
    pred_csv = data["disease_name"][index]
    print(pred_csv)

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/dtset2/disease_info.csv", encoding="cp1252")

In [ ]:
single_prediction("/content/drive/MyDrive/dtset2/test_images/Cheryy_Powdery_Mildew.JPG", model, data, device)

In [ ]:
single_prediction("/content/drive/MyDrive/dtset2/test_images/Apple_ceder_apple_rust.JPG")

In [ ]:
single_prediction("/content/drive/MyDrive/dtset2/test_images/Apple_scab.JPG")

In [ ]:
single_prediction("/content/drive/MyDrive/dtset2/test_images/Grape_esca.JPG")

In [ ]:
single_prediction("/content/drive/MyDrive/dtset2/test_images/apple_black_rot.JPG")

In [ ]:
single_prediction("/content/drive/MyDrive/dtset2/test_images/apple_healthy.JPG")

In [ ]:
single_prediction("/content/drive/MyDrive/dtset2/test_images/background_without_leaves.jpg")

In [ ]:
single_prediction("/content/drive/MyDrive/dtset2/test_images/blueberry_healthy.JPG")


In [ ]:
single_prediction("/content/drive/MyDrive/dtset2/test_images/cherry_healthy.JPG")

In [ ]:
single_prediction("/content/drive/MyDrive/dtset2/test_images/cherry_powdery_mildew.JPG")


In [ ]:
single_prediction("/content/drive/MyDrive/dtset2/test_images/corn_cercospora_leaf.JPG")

In [ ]:
single_prediction("/content/drive/MyDrive/dtset2/test_images/corn_common_rust.JPG")

In [ ]:
single_prediction("/content/drive/MyDrive/dtset2/test_images/corn_common_rust.JPG")


In [ ]:
single_prediction("/content/drive/MyDrive/dtset2/test_images/corn_healthy.jpg")


In [ ]:
single_prediction("/content/drive/MyDrive/dtset2/test_images/corn_northen_leaf_blight.JPG")



In [ ]:
single_prediction("/content/drive/MyDrive/dtset2/test_images/grape_black_rot.JPG")

In [ ]:
single_prediction("/content/drive/MyDrive/dtset2/test_images/grape_healthy.JPG")


In [ ]:
single_prediction("/content/drive/MyDrive/dtset2/test_images/grape_leaf_blight.JPG")


In [ ]:
single_prediction("/content/drive/MyDrive/dtset2/test_images/orange_haunglongbing.JPG")
